## Заполнение БД

In [ ]:
import pandas as pd

data = pd.read_excel('../../rutube_data/01_База_знаний.xlsx')
test_data = pd.read_excel('../../rutube_data/02_Реальные_кейсы.xlsx')

In [ ]:
from langchain_core.documents import Document

docs = []
for i, row in data.iterrows():
    page_content = row['Вопрос из БЗ']
    question = row['Вопрос из БЗ']
    metadata = {
        "answer": row['Ответ из БЗ'],
        "class_1": row["Классификатор 1 уровня"],
        "class_2": row["Классификатор 2 уровня"],
        "question": row['Вопрос из БЗ']
    }

    docs.append(Document(page_content, metadata=metadata))

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

In [ ]:
from langsmith import Client

client = Client()

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from langchain_community.storage import RedisStore
from langchain.retrievers import MultiVectorRetriever
from langchain_chroma import Chroma

byte_store = RedisStore(redis_url='redis://0.0.0.0:6379')
id_key = 'doc_id'
vecstore = Chroma(embedding_function=embeddings, persist_directory='./chroma_data', collection_name='rutube_base')

multi_retriever = MultiVectorRetriever(byte_store=byte_store, id_key=id_key, vectorstore=vecstore)

In [ ]:
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

In [ ]:
import json

with open('multi_questions.json', mode='r') as f:
    generated_questions = json.load(f)

In [ ]:
sub_docs = []

for doc, doc_id in zip(docs, doc_ids):
    doc.metadata[id_key] = doc_id
    _sub_docs = []
    for new_q in generated_questions.get(doc.metadata.get('question')):
        new_doc = Document(page_content=new_q, metadata={id_key: doc_id})
        _sub_docs.append(new_doc)
    sub_docs.extend(_sub_docs)
    sub_docs.append(doc)

In [ ]:
multi_retriever.vectorstore.add_documents(sub_docs)
multi_retriever.docstore.mset(list(zip(doc_ids, docs)))

In [ ]:
multi_retriever.search_kwargs = dict(k=1000)

In [ ]:
from tqdm import tqdm

total = len(test_data)
correct_results = 0
for i, row in tqdm(test_data.iterrows(), total=total):
    question = row['Вопрос пользователя']
    answer = row['Вопрос из БЗ']

    predicted = multi_retriever.invoke(question)
    if answer in [doc.metadata.get('question') for doc in predicted[:30]]:
        correct_results += 1

print(correct_results / total)

## Тестрование (end-to-end)

In [ ]:
import random
from typing import Dict, List


def create_examples(df) -> List[Dict[str, str]]:
    examples = []
    for i, row in df.iterrows():
        examples.append({
            "question": row['Вопрос пользователя'],
            "answer": row['Ответ сотрудника'],
            "class_1": row['Классификатор 1 уровня'],
            "class_2": row['Классификатор 2 уровня'],
            "answer_from_bz": row['Ответ из БЗ'],
        })

    return random.choices(examples, k=100)

In [ ]:
dataset_name = ""
dataset = client.read_dataset(dataset_name=dataset_name)

In [ ]:
examples = create_examples(test_data)

inputs, outputs = zip(
    *[({"question": row["question"]}, row) for row in examples]
)

In [ ]:
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

In [4]:
import requests


def predict(_inputs: dict) -> dict:
    response = requests.post('http://192.144.12.76:8080/predict', json=_inputs)
    return response.json()

In [5]:
predict({
  "question": "У меня ничего не открылось, хотя код отсканировала, помогите зарегистрироваться"
})

{'answer': 'К сожалению, я не могу найти соответствующую информацию, чтобы помочь вам с регистрацией на платформе RUTUBE. Могу ответить на вопросы, используя только данные из базы знаний. Если у вас есть проблемы со сканированием кода для регистрации, рекомендую убедиться, что качество сканирования хорошее и код введен без ошибок. Если проблема сохраняется, попробуйте обновить страницу или очистить кэш браузера. Если это не поможет, обратитесь в службу поддержки RUTUBE для получения дальнейшей помощи.',
 'class_1': None,
 'class_2': None,
 'docs': [],
 'total_docs': 0}

In [ ]:
from pydantic import BaseModel, Field


class EvalResponse(BaseModel):
    reason: str = Field(..., description="Причина выставления оценки")
    score: int = Field(..., description="Оценка ответа поданного на проверку")

### Наш сюдья

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
eval_system = """\
Вы учитель, который проверяет викторину.

Вам будут предоставлены ВОПРОС, ПРАВИЛЬНЫЙ ОТВЕТ и ОТВЕТ УЧЕНИКА.

Вот критерии оценки:

(1) Оценивайте ответы учащихся ТОЛЬКО на основе их фактической точности относительно правильного ответа.

(2) Убедитесь, что ответ ученика не содержит противоречивых утверждений.

(3) Допустимо, если ответ ученика содержит больше информации, чем правильный ответ, при условии, что она фактически точна относительно правильного ответа.

Оценка:

Оценка 1 означает, что ответ ученика соответствует всем критериям. Это самая высокая (лучшая) оценка.

Оценка 0 означает, что ответ ученика не соответствует всем критериям. Это самая низкая оценка, которую вы можете поставить.

Объясните свое рассуждение пошагово, чтобы убедиться, что ваши выводы корректны.

Избегайте просто указывать правильный ответ в самом начале."""

eval_human = """\
QUESTION: {question}
GROUND TRUTH ANSWER: {correct_answer}
STUDENT ANSWER: {student_answer}"""

In [ ]:
eval_prompt = ChatPromptTemplate.from_messages([
    ('system', eval_system),
    ('human', eval_human),
])

In [ ]:
def exact_match_class_1(run, example):
    return {"score": run.outputs["class_1"] == example.outputs["class_1"]}


def exact_match_class_2(run, example):
    return {"score": run.outputs["class_2"] == example.outputs["class_2"]}


def exact_match_bz_answer(run, example):
    return {"score": run.outputs["answer_from_bz"] == example.outputs["answer_from_bz"]}


def check_bz_answer_in_docs(run, example):
    answer_from_bz = example.outputs["answer_from_bz"]
    return {"score": answer_from_bz in run.outputs["docs"]}


def middle_num_docs(run, example):
    return {"score": run.outputs["total_docs"]}


def check_answer_correctness(run, example):
    model_with_so = model.with_structured_output(EvalResponse)
    evaluate_chain = eval_prompt | model_with_so
    result = evaluate_chain.invoke({
        "student_answer": run.outputs["answer"],
        "correct_answer": example.outputs["answer"],
        "question": example.inputs["question"],
    })
    return {"score": result.score, "description": result.reason, "key": "answer_score"}

In [ ]:
from langsmith import evaluate

evaluate(
    predict,
    data=dataset_name,
    evaluators=[check_bz_answer_in_docs, exact_match_class_2, exact_match_class_1, check_answer_correctness,
                middle_num_docs],
    metadata={"revision_id": "v.0.1.0"},
)

In [51]:
from langchain_community.storage import RedisStore

redis = RedisStore(redis_url="redis://0.0.0.0:6379")

In [52]:
for key in redis.yield_keys():
    print(key)

f1b6af58-9d7c-4cdb-affa-e6ee7dc771de
dbda9b51-b964-4534-bdec-286eca7b116b
9d33d8c2-a5d9-4a81-9f64-0764a479a519
c9fc08d4-74cd-4cb0-9b5a-3fe5b2734084
58fccb40-9ccd-436b-ba22-5bf7bb0a9ac3
d9be4aa9-a766-47d4-870f-276bf6665026
7b12cee5-b09e-4848-aaa2-e637c4a5c82a
d9f89b10-bb27-4d68-9872-e3117b915778
a0855937-1e79-449b-8b02-18f219b9bb11
5a5fe276-f829-4337-9288-b880316b8069
09267c13-ee21-45bb-8c7a-da6dc4f958b3
34bc2856-6d3a-4dd5-bb8f-092373c1ad8f
59efd891-b826-49f0-86b8-c2280269a690
82edbf4e-643c-47d6-816a-8a6df85e3886
72217701-3bb7-4b61-9e7c-71b6068ad174
12fa2865-7ff6-4d52-a799-8c13fba1f435
29e531b4-a739-4308-9ffe-442c3eb75a84
ec1133e6-adba-4b48-a2fa-b9939d4cf482
16f2104d-eedf-4405-98a5-e96c6d829a9f
b9fbf2a9-1e8a-4ce1-baa3-ccf5e147f437
68157558-b4c7-47b9-9d0a-d2d877b33e9a
e7b841d7-40da-4df3-8e98-9cff6b5e666e
d40fee29-9f14-44b5-8693-0b976c3a0999
1046e4be-2558-453b-8ebe-f926914e53cd
7fd9dd61-38b9-48c8-9e50-b18238345a7a
279f847d-bd29-4010-9e85-fac7750a1839
e21a8241-9ad2-4962-8cb9-8b420a414928
b

In [53]:
import requests


def check_api(url: str, data_json: dict = None) -> None:
    """
    Checks that response is in the correct JSON format.
    Args:
        url (str): The URL of the service to check.
        data_json (Optional[dict]): Data to post. Optional.
    """
    if data_json is None:
        data_json = {"question": "Как сменить пароль?"}
    elif not isinstance(data_json, dict):
        raise ValueError("The ``data_json`` must be in dict format.")
    elif not data_json.get("question", None):
        raise KeyError("The ``data_json'' must contain a ``question`` key.")
    resp = requests.post(url, json=data_json)
    resp.raise_for_status()
    answer_json = resp.json()
    if all(name in answer_json for name in ["answer", "class_1", "class_2"]):
        print("SUCCESSFUL. The service answer correctly.")
        print("Question: ", data_json['question'])
        print("Answer: ", answer_json['answer'])
        print("class_1: ", answer_json['class_1'])
        print("class_2: ", answer_json['class_2'])
    else:
        raise ValueError('The answer is not in the correct format. The expected format is '
                         '{"answer": "...", "class_1": "...", "class_2": "..."}.')


check_api("http://192.144.12.76:8080/predict")

SUCCESSFUL. The service answer correctly.
Question:  Как сменить пароль?
Answer:  Чтобы сменить пароль на RUTUBE, вам нужно авторизоваться на платформе, перейти в свой профиль по адресу https://rutube.ru/profile, и нажать на кнопку «Изменить пароль». После этого следуйте подсказкам на экране, чтобы установить новый пароль.
class_1:  УПРАВЛЕНИЕ АККАУНТОМ
class_2:  Персонализация
